<a href="https://colab.research.google.com/github/ERGOWHO/emberdatect_python_dataset/blob/main/dataset_handlemultiembers_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Step 1: initialize & Setup



## Step 1.1: Link to GOOGLE DRIVE




[reference](https://zhuanlan.zhihu.com/p/109634905)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step 1.2: Define folder paths

In [ ]:
## handle current folder path
import os
import re
import math
import json
import time 
import shutil
import cv2
from google.colab import output
import imageio
import os.path as osp


#modify here only
filename = 'scenes5_2ember_daytime'
PATH = '/content/drive/MyDrive/lab/embertrackingdataset/model_evaluation_tests/withforest_nofire/scenes5nofire/multiembers'
output_path = PATH + '/imgs/'
imgs_with_bbox_path = PATH + '/imgswithbbox/'

################
#input files:
################
file_path = PATH + '/' + filename + '.txt' # input data file
video_path = PATH + '/' + filename + 'ProceduralForest.avi'  # input video file
#mark_img_path = PATH+ '/' + filename + 'ProceduralForest_Moment.jpg' #input jpg

################
#Output files:
################
#marked_imgoutput_path = PATH + '/ember_straces.png' #
#check file already exists
folder1 = os.path.exists(output_path)
if not folder1: #create new folder if not exists
  os.makedirs(output_path) #video slip frams (imgs folder)
#folder2 = os.path.exists(imgs_with_bbox_path)
#if not folder2:
  #os.makedirs(imgs_with_bbox_path) #imgs with bbox for checking

path_jsonfile = PATH + '/' + filename + '.json'

path_x_ember = PATH + '/x_ember.txt'
path_y_ember = PATH + '/y_ember.txt'
path_id_ember = PATH + '/id_ember.txt'

####################
#get current time
####################

data_cap = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())



## Step 1.3:  Read data from UE4 log
#save data to list


In [ ]:
with open(file_path, "r", encoding="utf-8") as f:
  lines = f.readlines()

##Setting data from UE4, 1 frame 1 data, 
#if there is x ember co-exist at current frame, 
#there will be x data at the current frame
UE4_Life_time = 10
UE4_Loop_during = 2
UE4_Samplerate = 30 # unit: frames per sencond
Max_ember_onscreen = UE4_Life_time / UE4_Loop_during # the max number of embers on screen at same time
Frames_per_ember = UE4_Life_time * UE4_Samplerate
One_loop_frames = UE4_Loop_during * UE4_Samplerate # Start a new loop (generate a new ember) every 'One_loop_frames' frames 
#One_ember_lifetime_frames = UE4_Life_time / UE4_Loop_during * One_loop_frames


k = 0
num1 = 0
ScreenLoc = []
UniqueID = []
NormalizedAge = []
Loc_3D = []
Bbox_vertex_3D = []

for k in range(len(lines)-int(len(lines) % 3)):
  if num1 == 1:
    ScreenLoc.append(re.findall('[+-]?\d+\.?\d*',lines[k]))
  elif num1 == 2:
    NormalizedAge.append(re.findall('[+-]?\d+\.?\d*',lines[k]))
  else:
    num1 = 0
    UniqueID.append(re.findall('[+-]?\d+\.?\d*',lines[k]))
  num1 += 1

i=0
x_ember = []
y_ember = []
id_ember = []

scale_x = []
scale_y = []
scale_z = []
mesh_size = []

for i in range(len(ScreenLoc)):
  x_ember.append(float(ScreenLoc[i][2]))
  y_ember.append(float(ScreenLoc[i][3]))
  id_ember.append(float(UniqueID[i][2]))

  #mesh size scale          
  scale_x.append(float(NormalizedAge[i][2]))
  scale_y.append(float(NormalizedAge[i][3]))
  #scale_z.append(float(NormalizedAge[i][3]))

  #mesh_size.append(float(NormalizedAge[i][1]) * float(NormalizedAge[i][2]) * float(NormalizedAge[i][3]))
  mesh_size.append(float(NormalizedAge[i][1]) * float(NormalizedAge[i][2]))

num_ember = id_ember[i]

print("number of ember is : ", num_ember)
print("max scale.x: ", max(scale_x))
print("max scale.y: ", max(scale_y))

print("x_ember: ", x_ember)
print("y_ember: ", y_ember)
print("id_ember: ", id_ember)

print("scale.x: ", scale_x)
print("scale.y: ", scale_y)


number of ember is :  11.0
max scale.x:  2.518
max scale.y:  1.956
x_ember:  [563.16, 564.682, 566.201, 567.718, 569.233, 570.746, 572.257, 573.766, 575.272, 576.777, 578.279, 579.78, 581.278, 582.774, 584.269, 585.761, 587.251, 588.739, 590.225, 591.709, 593.191, 594.672, 596.15, 597.625, 599.099, 600.571, 602.041, 603.509, 604.975, 606.439, 607.901, 609.361, 610.819, 612.275, 613.729, 615.181, 616.631, 618.079, 619.525, 620.969, 622.411, 623.852, 625.29, 626.726, 628.161, 629.594, 631.024, 632.453, 633.879, 635.304, 636.727, 638.148, 639.567, 640.984, 642.4, 643.813, 645.225, 646.634, 648.042, 649.448, 650.852, 652.255, 541.998, 653.655, 544.155, 655.053, 546.31, 656.45, 548.461, 657.844, 550.61, 659.237, 552.755, 660.628, 554.897, 662.017, 557.036, 663.404, 559.172, 664.79, 561.304, 666.174, 563.434, 667.555, 565.561, 668.935, 567.685, 670.314, 569.806, 671.69, 571.923, 673.065, 574.038, 674.438, 576.149, 675.808, 578.258, 677.177, 580.363, 678.545, 582.466, 679.91, 584.565, 681.274

# Step 2: Handle Json file


## Step 2.1: Json "info" and "cate"

In [ ]:
#Info
info = []
inf = {}
inf["year"] = "2022"
inf["version"] = "1.0"
inf["descributor"] = "0508life2dur1emb1 pure forest scenes #3 background without fires or smoke, UE4 mesh scale setting: 1~2, screen size about 80*80 pixel, loop during 1, life time 2， 26 embers, 2 ember max at same time"
inf["data_created"] = data_cap
info.append(inf)

#CATEGORIES
category_id = 1
cate = []
cat = {}
cat["id"] = category_id
cat["name"] = "ember"
cate.append(cat)


## Step 2.2: get dataset imgs from video

In [ ]:


interval = 1  # 每间隔10帧取一张图片

#存img
imgarr = []

if __name__ == '__main__':
    num = 1
    video = cv2.VideoCapture(video_path)
    fps = video.get(cv2.CAP_PROP_FPS)#fps
    frames = video.get(cv2.CAP_PROP_FRAME_COUNT)#total frame
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print("fps=",int(fps),"frames=",int(frames))

    while video.isOpened():
        is_read, frame = video.read()
        if is_read:
          if num % interval == 0:
            #if num % interval == 1:
            file_name = '%d' % num
            cv2.imwrite(output_path + str(file_name) + '.jpg', frame)#输出图片
            #cv2.imwrite(imgs_with_bbox_path + str(file_name) + '.jpg', frame)#输出图片for bbox check
            #cv2.waitKey(1)
            img = {}
            img["id"] = num
            img["width"] = width
            img["height"] = height
            img["file_name"] = str(file_name) + ".jpg"
            img["date_captured"] = data_cap
            #save to one dict
            imgarr.append(img)
            print(num)
          num += 1
        else:
          break

output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

shutil.copytree(output_path, imgs_with_bbox_path)
print('copy dir finished!')

# End of code alert
# Play an audio beep. Any audio URL will do.

output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
cv2.waitKey(1)
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

fps= 30 frames= 690
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272


## Step 2.3: (For multi embers) handle bboxes


1.   Map ids to embers 
2.   Add bbox for imgs according to each data

*   仅输出含有bbox的图像/ out put imgs only when it has bbox
*   在每个有bbox的图像上标注/mark bbox for checking进块

3.   Generate ember[ ] for json 




In [ ]:
#存ember
embarr = []

x=x_ember
y=y_ember
ember_number = 2
#ue4 mesh size VS pixel ratio adjust
pixel_mesh_scale_ratio = 17

#对于data中每个数据来说
for i in range(len(x_ember)):
  
  x[i]=round(x[i])
  y[i]=round(y[i])

  #if the current ember is in the frame:
  if 0 < x[i] < 2560 and 0 < y[i] < 1080: 
    ############## make json ember file
    #calculate the bbox
    bbx_width = pixel_mesh_scale_ratio * scale_x[i]
    bbx_height = pixel_mesh_scale_ratio * scale_x[i]
    area = bbx_width * bbx_height 
    # top left coordinate of the current ember
    x_top_left = int(x[i] - bbx_width/2)
    y_top_left = int(y[i] - bbx_height/2)
    bbox = [x_top_left, y_top_left, bbx_width, bbx_height]
    #for each ember data, turn data sequence to correspond frames:
    if i <= Frames_per_ember/2:
      image_id = i+1
    else:
      image_id = math.ceil((i+1 - Frames_per_ember/2) /2 + Frames_per_ember/2) # /2 if ember = 2
    #json.ember{}
    ember = {}
    ember["area"] = area
    ember["image_id"] = image_id
    ember["bbox"] = bbox
    ember["id"] = 1000000 * int(id_ember[i]) + 10000000000 + i
    ember["ember_id"] = int(id_ember[i])
    ember["category_id"] = category_id
    embarr.append(ember)

    ########## mark bbox on graph for checking:

    #打开本数据对应的img  
    image = cv2.imread(imgs_with_bbox_path + str(image_id) + '.jpg')
    #将本数据的ember坐标中心标注在对应的图上
    cv2.circle(image,(x[i],y[i]),1,(255,255,255),-1)

    #get vertex
    x_bottom_right = int(x[i] + bbx_width/2)
    y_bottom_right = int(y[i] + bbx_height/2)

    #The function cv::rectangle draws a rectangle outline or a filled rectangle whose two opposite corners are pt1 and pt2.
    draw_bbox = cv2.rectangle(image,(x_top_left,y_top_left),(x_bottom_right, y_bottom_right),(255,255,255),1)

    cv2.imwrite(imgs_with_bbox_path + str(image_id) + '.jpg', draw_bbox) 

    
# End of code alert
# Play an audio beep. Any audio URL will do.
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
cv2.waitKey(1)
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
cv2.waitKey(1)
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

# Step 3: Pick up all the contains for json file + Make output files 


In [ ]:
j_info = json.dumps(info,ensure_ascii=False)
j_imges = json.dumps(imgarr,ensure_ascii=False)
j_categories = json.dumps(cate,ensure_ascii=False)
j_annotations = json.dumps(embarr,ensure_ascii=False)

print("info"+j_info)
print("img"+j_imges)
print("cat"+j_categories)
print("ann"+j_annotations)

with open(path_jsonfile, "w") as f:
 f.write("{")
 f.write(''' "info": ''')
 f.write(j_info)
 f.write(", ") 
 f.write('''"images": ''')
 f.write(j_imges)
 f.write(", ")
 f.write('''"categories": ''')
 f.write(j_categories)
 f.write(", ")
 f.write('''"annotations": ''')
 f.write(j_annotations)
 f.write(", ")
 f.write("}")

print("x_ember: ", x_ember)
print("y_ember: ", y_ember)
print("id_ember: ", id_ember)
#print("age_ember: ", age_ember)

with open(path_x_ember, "w") as f:
 f.write("x_ember:")
 f.write(str(x_ember))
with open(path_y_ember, "w") as f:
 f.write("y_ember:")
 f.write(str(y_ember))
with open(path_id_ember, "w") as f:
 f.write("id_ember:")
 f.write(str(id_ember))
#with open("/content/drive/MyDrive/lab/embertrackingdataset/0414take2dataset/age_ember.txt", "w") as f:
 #f.write("age_ember:")
 #f.write(str(age_ember))

info[{"year": "2022", "version": "1.0", "descributor": "0508life2dur1emb1 pure forest scenes #3 background without fires or smoke, UE4 mesh scale setting: 1~2, screen size about 80*80 pixel, loop during 1, life time 2， 26 embers, 2 ember max at same time", "data_created": "2022-05-21 21:31:14"}]
img[{"id": 1, "width": 2560, "height": 1080, "file_name": "1.jpg", "date_captured": "2022-05-21 21:31:14"}, {"id": 2, "width": 2560, "height": 1080, "file_name": "2.jpg", "date_captured": "2022-05-21 21:31:14"}, {"id": 3, "width": 2560, "height": 1080, "file_name": "3.jpg", "date_captured": "2022-05-21 21:31:14"}, {"id": 4, "width": 2560, "height": 1080, "file_name": "4.jpg", "date_captured": "2022-05-21 21:31:14"}, {"id": 5, "width": 2560, "height": 1080, "file_name": "5.jpg", "date_captured": "2022-05-21 21:31:14"}, {"id": 6, "width": 2560, "height": 1080, "file_name": "6.jpg", "date_captured": "2022-05-21 21:31:14"}, {"id": 7, "width": 2560, "height": 1080, "file_name": "7.jpg", "date_captur

# Step 4：make a gif

In [ ]:

def img2gif(img_dir, gif_path, duration):
    """

    :param img_dir: 包含图片的文件夹
    :param gif_path: 输出的gif的路径
    :param duration: 每张图片切换的时间间隔，与fps的关系：duration = 1 / fps
    :return:
    """
    frames = []
    for idx in sorted(os.listdir(img_dir)):
        img = osp.join(img_dir, idx)
        frames.append(imageio.imread(img))

    imageio.mimsave(gif_path, frames[100:130], 'GIF', duration=duration)
    print('Finish changing!')


if __name__ == '__main__':
    gif_path = osp.join(PATH, 'output.gif')
    img2gif("/content/drive/MyDrive/lab/embertrackingdataset/model_evaluation_tests/withforest_nofire/scenes5nofire/multiembers/2embers/imgswithbbox", gif_path=gif_path, duration=0.1)

# End of code alert
# Play an audio beep. Any audio URL will do.
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
cv2.waitKey(2)
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
cv2.waitKey(5)
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
cv2.waitKey(2)
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

Finish changing!


#test


In [ ]:
def accSum(n):
    sum = 0
    for i in range(1, n+1): #[1,n+1)
        sum += i
    return sum

print(accSum(5))

15


In [ ]:
UE4_Life_time = 10
UE4_Loop_during = 2
UE4_Samplerate = 30 # unit: frames per sencond
Max_ember_onscreen = UE4_Life_time / UE4_Loop_during # the max number of embers on screen at same time
Frames_per_ember = UE4_Life_time * UE4_Samplerate
One_loop_frames = UE4_Loop_during * UE4_Samplerate # Start a new loop (generate a new ember) every 'One_loop_frames' frames 
#One_ember_lifetime_frames = UE4_Life_time / UE4_Loop_during * One_loop_frames

print("Max_ember_onscreen",Max_ember_onscreen)
print("Frames_per_ember",Frames_per_ember)
print("One_loop_frames",One_loop_frames)
print("One_ember_lifetime_frames",One_ember_lifetime_frames)

Max_ember_onscreen 5.0
Frames_per_ember 300
One_loop_frames 60
One_ember_lifetime_frames 300.0
